In [26]:
import pandas as pd
import numpy as np
from sklearn.metrics import pairwise_distances as distance
import json

In [97]:
stand_csv = pd.read_csv("stand.csv")
stand_csv["keypoint_coords"] = stand_csv["keypoint_coords"].apply(lambda x : json.loads(x))
stand_csv["keypoint_scores"] = stand_csv["keypoint_scores"].apply(lambda x : json.loads(x))
stand_csv

,Unnamed: 0,keypoint_coords,keypoint_scores
0,0,"[[41.17484540717547, 424.118377831564], [18.37...","[0.9492308497428894, 0.9189706444740295, 0.951..."
1,1,"[[65.42472422400186, 418.31286562686904], [31....","[0.960146427154541, 0.8650374412536621, 0.9544..."
2,2,"[[62.49335976534112, 408.9349744878888], [32.9...","[0.9485349059104919, 0.8589192628860474, 0.948..."
3,3,"[[59.07950964639353, 403.96868692060406], [35....","[0.9301652908325195, 0.8851189613342285, 0.948..."
4,4,"[[57.207370048345524, 397.7698863636364], [33....","[0.9177314043045044, 0.8564090132713318, 0.945..."
...,...,...,...
2318,2318,"[[82.09326500116393, 988.3409228165184], [12.7...","[0.818300724029541, 0.3718859553337097, 0.7361..."
2319,2319,"[[93.70561102933662, 986.9426278292277], [31.7...","[0.8564101457595825, 0.5420486927032471, 0.797..."
2320,2320,"[[78.38059891101926, 1002.6481942354777], [-5....","[0.7710866928100586, 0.16072413325309753, 0.71..."
2321,2321,"[[53.604439668877184, 997.8381508275082], [-13...","[0.760976254940033, 0.1255340278148651, 0.6220..."


In [98]:
crunch_csv = pd.read_csv("crunch.csv")
crunch_csv["keypoint_coords"] = crunch_csv["keypoint_coords"].apply(lambda x : list(json.loads(x)))
crunch_csv["keypoint_scores"] = crunch_csv["keypoint_scores"].apply(lambda x : list(json.loads(x)))
crunch_csv

,Unnamed: 0,keypoint_coords,keypoint_scores
0,0,"[[81.94267601190612, 416.8615109384345], [38.4...","[0.9523769617080688, 0.8400066494941711, 0.952..."
1,1,"[[74.69318425378134, 413.80083586040297], [37....","[0.9642161130905151, 0.8565571308135986, 0.964..."
2,2,"[[78.44337791620299, 403.7170512368234], [36.5...","[0.9657862186431885, 0.8683214783668518, 0.960..."
3,3,"[[81.32825807083485, 397.2836044530549], [36.7...","[0.9633383750915527, 0.874426007270813, 0.9602..."
4,4,"[[78.80350529870321, 398.26694470273253], [40....","[0.9723122119903564, 0.8743901252746582, 0.966..."
...,...,...,...
4352,4352,"[[15.047931338465492, 746.429168847189], [-43....","[0.27085429430007935, 0.2919919788837433, 0.30..."
4353,4353,"[[14.062564872031988, 732.8876228880084], [-56...","[0.30790072679519653, 0.3133243918418884, 0.36..."
4354,4354,"[[13.013681666795598, 732.6332961087021], [-52...","[0.4788801670074463, 0.5108022093772888, 0.330..."
4355,4355,"[[-8.970345574756001, 731.7963376569976], [-58...","[0.4029550552368164, 0.46766090393066406, 0.29..."


In [99]:
def extract_feature(keypoint_scores, keypoint_coords, SCORE_THRESHOLD=0.15):
    features = distance(keypoint_coords[0:1], keypoint_coords[1:])[0]
    for index in range(1, len(keypoint_scores)):
        if keypoint_scores[index] < SCORE_THRESHOLD:
            features[index - 1] = 0
    # normalize
    mx = max(features)
    for index in range(len(features)):
        features[index] = features[index] / mx
    return features

# derives features from keypoints
stand_csv["features"] = stand_csv.apply(lambda row: extract_feature(row["keypoint_scores"], row["keypoint_coords"]), axis=1)
# derives features from keypoints
crunch_csv["features"] = crunch_csv.apply(lambda row: extract_feature(row["keypoint_scores"], row["keypoint_coords"]), axis=1)


/Users/tadangvinhphuc/Library/Python/3.6/lib/python/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':
/Users/tadangvinhphuc/Library/Python/3.6/lib/python/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


In [102]:
# concat 2 df
# append features and label to the numpy
# train_x = np.concatenate(stand_csv["features"], crunch_csv["features"])

# build 2-classes data
ds_x = []
ds_y = []
for _, row in stand_csv.iterrows():
    ds_x.append(list(row["features"]))
    ds_y.append(0)
for _, row in crunch_csv.iterrows():
    ds_x.append(list(row["features"]))
    ds_y.append(1)

In [103]:

from sklearn.model_selection import train_test_split
ds_x = np.array(ds_x)
ds_y = np.array(ds_y)

# train test split
train_x, test_x, train_y, test_y = train_test_split(ds_x, ds_y, test_size = 1/3, random_state=0) # use random state for recover result with ease

In [121]:
# preprocess NaN values
selected_non_nan = ~np.isnan(train_x).any(axis=1)
train_x = train_x[selected_non_nan] # each row 
train_y = train_y[selected_non_nan]

In [122]:
from sklearn.svm import SVC

clf = SVC()
clf.fit(train_x, train_y)

SVC()

In [124]:
selected_non_nan = ~np.isnan(test_x).any(axis=1)
test_x = test_x[selected_non_nan] # each row 
test_y = test_y[selected_non_nan]

In [128]:
pred_y = clf.predict(test_x)
pred_y

array([1, 1, 1, ..., 1, 1, 1])

In [127]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, pred_y)

(array([0, 1]), array([ 156, 2070]))